In [17]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, concatenate, Dropout, Reshape, Permute, Activation, ZeroPadding2D, Cropping2D, Add
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Conv2DTranspose, DepthwiseConv2D, Flatten, Dense, BatchNormalization, LeakyReLU
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [ ]:
IMG_HEIGHT = 256
IMG_WIDTH = 256
BATCH_SIZE = 16

In [ ]:
x_train = np.load('data/x_train.npz')['data'].astype(np.float32)
y_train = np.load('data/y_train.npz')['data'].astype(np.float32)
x_val = np.load('data/x_val.npz')['data'].astype(np.float32)
y_val = np.load('data/y_val.npz')['data'].astype(np.float32)

print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    brightness_range=[0.5, 1.3]
)

val_datagen = ImageDataGenerator(
    rescale=1./255
)

train_gen = train_datagen.flow(
    x_train,
    y_train,
    batch_size=BATCH_SIZE,
    shuffle=True
)

val_gen = val_datagen.flow(
    x_val,
    y_val,
    batch_size=BATCH_SIZE,
    shuffle=False
)

In [27]:
# # 깃허브 긁어 온거
# inputs = Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))

# # encode
# conv1 = Conv2D(32, 3, activation='relu', padding='same', kernel_initializer='orthogonal')(inputs)
# # conv1 = Dropout(0.2)(conv1)
# conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='orthogonal')(conv1)
# pool1 = MaxPooling2D(pool_size=2)(conv1)
# # (128, 128, 64)

# shortcut_1 = pool1

# conv2 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='orthogonal')(pool1)
# # conv2 = Dropout(0.2)(conv2)
# conv2 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='orthogonal')(conv2)
# pool2 = MaxPooling2D(pool_size=2)(conv2)
# # (64, 64, 128)

# shortcut_2 = pool2

# conv3 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='orthogonal')(pool2)
# # conv3 = Dropout(0.2)(conv3)
# conv3 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='orthogonal')(conv3)
# pool3 = MaxPooling2D(pool_size=2)(conv3)
# # (32, 32, 256)

# # middle depthwiseconv nodes
# shortcut_3 = pool3

# mid = DepthwiseConv2D(3, activation='relu', padding='same', kernel_initializer='orthogonal')(pool3)
# mid = Conv2D(256, 1, activation='relu', padding='same', kernel_initializer='orthogonal')(mid)

# mid = DepthwiseConv2D(3, activation='relu', padding='same', kernel_initializer='orthogonal')(mid)
# mid = Conv2D(256, 1, activation='relu', padding='same', kernel_initializer='orthogonal')(mid)

# mid = Add()([shortcut_3, mid])

# # decode
# up8 = UpSampling2D(size=2)(mid)
# up8 = concatenate([up8, conv3], axis=-1)
# conv8 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='orthogonal')(up8)
# # conv8 = Dropout(0.2)(conv8)
# conv8 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='orthogonal')(conv8)
# # (64, 64, 128)

# conv8 = Add()([shortcut_2, conv8])

# up9 = UpSampling2D(size=2)(conv8)
# up9 = concatenate([up9, conv2], axis=-1)
# conv9 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='orthogonal')(up9)
# # conv9 = Dropout(0.2)(conv9)
# conv9 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='orthogonal')(conv9)
# # (128, 128, 64)

# conv9 = Add()([shortcut_1, conv9])

# up10 = UpSampling2D(size=2)(conv9)
# up10 = concatenate([up10, conv1], axis=-1)
# conv10 = Conv2D(32, 3, activation='relu', padding='same', kernel_initializer='orthogonal')(up10)
# # conv10 = Dropout(0.2)(conv10)
# conv10 = Conv2D(32, 3, activation='relu', padding='same', kernel_initializer='orthogonal')(conv10)
# # (256, 256, 32)

# conv11 = Conv2D(2, 1, padding='same', activation='relu',
#                 kernel_initializer='he_normal', kernel_regularizer=l2(0.005))(conv10)
# conv11 = Reshape((IMG_HEIGHT * IMG_WIDTH, 2))(conv11)
# # (256, 256, 2)

# conv11 = Activation('softmax')(conv11)

# outputs = Reshape((IMG_HEIGHT, IMG_WIDTH, 2))(conv11)

# model = Model(inputs=inputs, outputs=outputs)

# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# model.summary()

Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_22 (InputLayer)           [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d_184 (Conv2D)             (None, 256, 256, 32) 896         input_22[0][0]                   
__________________________________________________________________________________________________
conv2d_185 (Conv2D)             (None, 256, 256, 64) 18496       conv2d_184[0][0]                 
__________________________________________________________________________________________________
max_pooling2d_35 (MaxPooling2D) (None, 128, 128, 64) 0           conv2d_185[0][0]                 
____________________________________________________________________________________________

In [38]:
# Unet 아키텍쳐 CNN 모델
def UNet(pretrained_weights = None,input_size = (256,256,3)):
    inp = Input(input_size)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inp)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'sigmoid', padding = 'same', kernel_initializer = 'he_normal')(conv9)

    model = Model(inputs = inp, outputs=[conv9])

    return model

In [39]:
model = UNet()
model.summary()
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

Model: "model_15"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_28 (InputLayer)           [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d_318 (Conv2D)             (None, 256, 256, 64) 1792        input_28[0][0]                   
__________________________________________________________________________________________________
conv2d_319 (Conv2D)             (None, 256, 256, 64) 36928       conv2d_318[0][0]                 
__________________________________________________________________________________________________
max_pooling2d_58 (MaxPooling2D) (None, 128, 128, 64) 0           conv2d_319[0][0]                 
___________________________________________________________________________________________

In [ ]:
history = model.fit_generator(train_gen,
    validation_data=val_gen,
    epochs=100,
    callbacks=[
        ModelCheckpoint('models/masing_model.h5', monitor='val_loss', save_best_only=True, verbose=1),
        ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=10, verbose=1, min_lr=1e-05)
    ]
)